# Evaluating model and interpret the classification results

In [ ]:
import torchray
import torch
import attr
from pathlib import Path
from torchvision import datasets, transforms
from torchray.benchmark import get_example_data, plot_example
from torchvision import models
import torchvision
import matplotlib.pyplot as plt

In [ ]:
import cv2

In [ ]:
!pip install opencv-python

In [ ]:
a = cv2.imread('../data/figures/test/abnormal/1_0_102.png')

In [ ]:
a.dtype

In [ ]:
plt.imshow(a)

In [ ]:
@attr.s(auto_attribs=True)
class DataPreparation:
    data_dir: Path
    device: str = attr.ib(default=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), init=False)

    @staticmethod
    def data_transformations():
        data_transforms = {
            'test': transforms.Compose([
                transforms.Resize((224,224)),
        #         transforms.CenterCrop(800),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
        return data_transforms
    
    def create_dataloaders(self, batch_size, shuffle, num_workers):
        data_transforms = self.data_transformations()
        
        image_datasets = {
            'test': datasets.ImageFolder(self.data_dir, data_transforms['test'])
        }
        dataloaders = {
            'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        }
        dataset_sizes = {
            'test': len(image_datasets['test'])
        }
        return dataloaders, dataset_sizes


In [ ]:
data_prep = DataPreparation('../data/figures/test')
data, size = data_prep.create_dataloaders(4, False, 4)

In [ ]:
model_path = Path().cwd().parents[0] / "models/resnet50_d_27_t_14_15.pth"

In [ ]:
model = torch.load(model_path)
model.eval();

In [ ]:
for inputs, labels in data['test']:
    inputs = inputs.to(0)
    labels = labels.to(0)
    x = inputs
    print(x.size(), labels.size())
    break

In [ ]:
from torchray.attribution.common import Probe, get_module
from torchray.attribution.grad_cam import grad_cam
from torchray.attribution.grad_cam import gradient_to_grad_cam_saliency

In [ ]:
saliency = grad_cam(model, x, 20, saliency_layer='features.29')

In [ ]:
def deprocess(image):
    transform = transforms.Compose([
        transforms.Normalize(mean=[0, 0, 0], std=[4.3668, 4.4643, 4.4444]),
        transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
        transforms.ToPILImage(),
    ])
    return transform(image)

def show_img(PIL_IMG):
    plt.imshow(np.asarray(PIL_IMG))

In [ ]:
x.size()
print(x)

In [ ]:
x.requires_grad_();
scores = model(x)

In [ ]:
scores

In [ ]:
score_max_index = scores.argmax(dim=1)
print(f"SCORE MAX INDEX: {score_max_index}")
score_max = scores[0,score_max_index]
print(f"SCORE MAX: {score_max}")

'''
backward function on score_max performs the backward pass in the computation graph and calculates the gradient of 
score_max with respect to nodes in the computation graph
'''
score_max.backward(torch.ones_like(score_max))
# score_max.backward()

'''
Saliency would be the gradient with respect to the input image now. But note that the input image has 3 channels,
R, G and B. To derive a single class saliency value for each pixel (i, j),  we take the maximum magnitude
across all colour channels.
'''
saliency, _ = torch.max(x.grad.data.abs(),dim=1)

In [ ]:
# code to plot the saliency map as a heatmap
plt.figure()
img1 = plt.imshow(saliency[0].cpu().numpy(), cmap=plt.cm.hot, alpha=.7)
plt.axis('off')
plt.show()
img2 = plt.imshow(deprocess(x[0].cpu()))
plt.axis('off')
plt.show()

In [ ]:
saliency.mean()

In [ ]:
m = models.alexnet(pretrained=True)